In [1]:
import torch 
from ipsl_dataset import input_variables,IPSL_DCPP
import numpy as np

/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = IPSL_DCPP('train')
train_dataloader = torch.utils.data.DataLoader(train,batch_size=1,shuffle=True)



def z_normalize(data:np.ndarray,axes:tuple):
    mean = np.nanmean(data,axis=axes,keepdims=False)
    std_dev = np.nanstd(data,axis=axes,keepdims=False)
    # mean = np.broadcast_to(mean,data.shape)
    # std_dev = np.broadcast_to(std_dev,data.shape)
    # standardized_data = (data - mean) / std_dev
    return mean,std_dev

8it [00:07,  1.10it/s]


In [3]:
means = []
std_devs = []
# datas = []
for count in range(1000):
    # x,y,t1,t2 = batch
    batch = next(iter(train_dataloader))
    x = batch['inputs'].squeeze()
    # datas.append(x)
    
    mean,std_dev = z_normalize(x.numpy(),(1,2))
    means.append(mean)
    std_devs.append(std_dev)
fin_means = np.nanmean(np.array(means),axis=0)
fin_stds = np.nanmean(np.array(std_devs),axis=0)

In [4]:
np.save('means',fin_means)
np.save('std_devs',fin_stds)

In [4]:
stacked = np.stack(datas)

In [5]:
stacked.shape

(10, 136, 143, 144)

In [6]:
np.array(means).shape

(10, 136)

In [7]:
np.load('std_devs.npy')

array([2.74373341e+01, 3.53500099e+01, 1.34439974e+01, 9.45808664e-02,
       5.02852321e-01, 4.06214863e-01, 1.13858409e-01, 4.71004695e-02,
       7.52755404e-01, 4.18635346e-02, 1.13169622e+00, 1.69927371e+00,
       3.58024955e+00, 1.51866484e+01, 8.21689415e+00, 7.27038878e-06,
       6.37716630e-06, 1.86779681e-09, 4.40111370e-09, 1.05015019e-08,
       2.81162702e-08, 2.13032646e+01, 1.52532840e+00, 1.27155045e+02,
       3.34994656e-05, 2.01305520e-05, 1.67795288e+02, 7.84410226e-09,
       1.27610118e-08, 2.49755883e-09, 2.34500463e-09, 6.31195540e-09,
       6.37740095e-06, 5.82076609e-09, 1.12430367e-08, 0.00000000e+00,
       9.02046260e-09, 8.78637456e-06, 2.78358631e+01, 1.61617165e+01,
       2.15115795e+01, 3.34921926e-01, 1.75342914e-02, 2.54790955e+01,
       8.71168673e-02, 2.47866083e-05, 6.18532219e+01, 2.56314068e+01,
       1.45131426e+01, 5.78049384e-03, 3.50952505e-05, 2.59288972e-05,
       8.72585588e-06, 1.48088808e+01, 9.52940137e+03, 1.06032434e+03,
      

In [6]:
np.nanmean(np.array(std_devs),axis=0)

array([0.52326447, 0.52326846, 0.5230063 , 0.52335125, 0.52302456,
       0.5229055 , 0.52360886, 0.5251202 , 0.52319324, 0.5239608 ,
       0.5232958 , 0.52326167, 0.5233174 , 0.52302617, 0.52371144,
       0.5406014 , 0.5186203 , 0.50004727, 0.5251411 , 0.52808094,
       0.5351337 , 0.5232961 , 0.5255908 , 0.5273697 , 0.5244022 ,
       0.53295463, 0.522011  , 0.54562515, 0.5389302 , 0.5420583 ,
       0.5389902 , 0.52543753, 0.51862913, 0.5379356 , 0.5279912 ,
       0.53130925, 0.        , 0.52594286, 0.5420642 , 0.5233633 ,
       0.52337706, 0.523335  , 0.9988474 , 0.9997422 , 1.0060253 ,
       1.0011673 , 1.0010872 , 1.0010636 , 0.9982065 , 1.0025305 ,
       0.99903685, 0.9986113 , 0.99641967, 1.0019749 , 0.9987043 ,
       0.99968874, 0.98943317, 1.0005232 , 0.9998635 , 1.0001446 ,
       1.0004922 , 1.000277  , 1.0189067 , 1.0189686 , 1.0172659 ,
       1.0396899 , 1.0384327 , 1.0331126 , 1.0372727 , 1.0090816 ,
       0.9992389 , 1.0001417 , 1.0010449 , 0.9993535 , 0.99305

In [38]:
example = np.array(std_devs)[:,0]

In [39]:
ns = np.zeros(10) 

In [40]:
ns[:] = 20000

In [41]:
np.power((((ns - 1) * np.power(example, 2)).sum() / (ns-1).sum()),1/2)

27.434968050351873

In [42]:
def pooled_standard_deviation(std_devs, sample_sizes):
    # Check if the lengths of std_devs and sample_sizes are the same
    if len(std_devs) != len(sample_sizes):
        raise ValueError("The lengths of std_devs and sample_sizes must be the same")

    # Calculate the pooled standard deviation
    numerator = np.sum((sample_sizes - 1) * np.square(std_devs))
    denominator = np.sum(sample_sizes - 1)
    pooled_std_dev = np.sqrt(numerator / denominator)

    return pooled_std_dev


In [43]:
pooled_standard_deviation(example,ns)

27.43496860653218

In [37]:
# np.nanmean(stacked,axis=(0,2,3),keepdims=True).flatten()
np.nanstd(stacked,axis=(0,2,3),keepdims=True).flatten()

array([2.74349670e+01, 3.53473053e+01, 1.34683065e+01, 9.47383642e-02,
       5.02782524e-01, 4.06380713e-01, 1.13592565e-01, 4.65145446e-02,
       7.52374947e-01, 4.19500917e-02, 1.13189268e+00, 1.69919646e+00,
       3.57755399e+00, 1.52129622e+01, 8.29440784e+00, 7.95205506e-06,
       6.40511462e-06, 1.48377366e-09, 4.27330615e-09, 1.02439026e-08,
       3.02476941e-08, 2.12974091e+01, 1.54097509e+00, 1.28200745e+02,
       3.35349650e-05, 2.22251801e-05, 1.67519547e+02, 9.09886211e-09,
       1.43189824e-08, 2.83535861e-09, 2.58306687e-09, 6.54988241e-09,
       6.40512144e-06, 6.45606368e-09, 1.13663017e-08, 1.69795733e-08,
       0.00000000e+00, 8.78796591e-09, 9.70986366e-06, 2.78150291e+01,
       1.61476822e+01, 2.14998188e+01, 3.32811952e-01, 1.77256074e-02,
       2.57896557e+01, 8.73388276e-02, 2.49866243e-05, 6.23638878e+01,
       2.68216534e+01, 1.48464708e+01, 5.70764393e-03, 3.52693351e-05,
       2.63229613e-05, 8.38321102e-06, 1.45853186e+01, 9.49853418e+03,
      

In [10]:
np.load('means.npy')

array([ 1.5295983e+01,  5.9352486e+01,  6.8814673e+00,  7.3568985e-02,
        6.8271554e-01,  5.1240999e-01,  1.7030577e-01,  1.4266686e-02,
        4.8553792e-01,  5.7603046e-02,  1.4806440e+00,  2.1906912e+00,
        2.2179601e+00,  7.9244843e+00,  1.9737396e+01,  7.0135538e-06,
        2.9890734e-06,  7.7444451e-10,  3.9589851e-09,  8.0952711e-09,
        2.1907553e-08,  2.6994478e+01,  1.2580804e+00,  5.3451168e+01,
        1.2795696e-05,  4.3844843e-06,  4.8505786e+02,  1.3336544e-10,
        9.2136467e-09,  1.9803790e-09,  1.9572637e-09,  3.8131547e-09,
        2.9890864e-06, -4.3947952e-09, -8.2991152e-09,  1.2693889e-08,
        0.0000000e+00,  8.1114715e-09,  6.6337343e-06,  3.1314884e+01,
        1.4469756e+01,  1.6845201e+01,  6.3567930e-01,  2.1495221e-02,
        6.8154793e+01,  8.2375333e-02,  2.5663441e-05,  6.4373619e+01,
        1.4357220e+01,  8.2628647e+01,  7.1226372e-03,  2.8216795e-05,
        1.2747617e-05,  4.1906696e-06,  1.7807863e+01,  9.6571016e+04,
      

In [5]:
for count in range(10):
    # x,y,t1,t2 = batch
    batch = next(iter(train_dataloader))
    print(batch['inputs'])

torch.Size([136, 143, 144])
(136, 1, 1)


/gpfsdswork/projects/rech/mlr/udy16au/ipsl_dataset.py:192: RuntimeWarning: invalid value encountered in divide
  inputs = (inputs - self.means) / self.stds
/gpfsdswork/projects/rech/mlr/udy16au/ipsl_dataset.py:193: RuntimeWarning: invalid value encountered in divide
  targets = (targets - self.means) / self.stds


tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         ...,

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 

/gpfsdswork/projects/rech/mlr/udy16au/ipsl_dataset.py:192: RuntimeWarning: invalid value encountered in divide
  inputs = (inputs - self.means) / self.stds
/gpfsdswork/projects/rech/mlr/udy16au/ipsl_dataset.py:193: RuntimeWarning: invalid value encountered in divide
  targets = (targets - self.means) / self.stds


torch.Size([136, 143, 144])
(136, 1, 1)
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         ...,

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ..

...

/gpfsdswork/projects/rech/mlr/udy16au/ipsl_dataset.py:192: RuntimeWarning: invalid value encountered in divide
  inputs = (inputs - self.means) / self.stds
/gpfsdswork/projects/rech/mlr/udy16au/ipsl_dataset.py:193: RuntimeWarning: invalid value encountered in divide
  targets = (targets - self.means) / self.stds


torch.Size([136, 143, 144])
(136, 1, 1)
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         ...,

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ..

/gpfsdswork/projects/rech/mlr/udy16au/ipsl_dataset.py:192: RuntimeWarning: invalid value encountered in divide
  inputs = (inputs - self.means) / self.stds
/gpfsdswork/projects/rech/mlr/udy16au/ipsl_dataset.py:193: RuntimeWarning: invalid value encountered in divide
  targets = (targets - self.means) / self.stds


torch.Size([136, 143, 144])
(136, 1, 1)
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         ...,

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ..

/gpfsdswork/projects/rech/mlr/udy16au/ipsl_dataset.py:192: RuntimeWarning: invalid value encountered in divide
  inputs = (inputs - self.means) / self.stds
/gpfsdswork/projects/rech/mlr/udy16au/ipsl_dataset.py:193: RuntimeWarning: invalid value encountered in divide
  targets = (targets - self.means) / self.stds
